# Jarvis-AI Voice Assistant

## 1. Import Necessary Libraries

In [18]:

import speech_recognition as sr
import pyttsx3
import requests
import time
import spacy
from gtts import gTTS
import os



## 2. Set Up Environment for Text-to-Speech

In [19]:

def setup_tts_engine():
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[1].id)  # You can change index for different voices
    engine.setProperty('rate', 150)  # Speed of speech
    return engine

def speak_text(engine, text):
    engine.say(text)
    engine.runAndWait()



## 3. Capture Voice Input

In [20]:
def capture_voice_input(recognizer, microphone):
    with microphone as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    return audio

def recognize_speech(recognizer, audio):
    try:
        text = recognizer.recognize_google(audio)
        print(f"Recognized: {text}")
        return text
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

## 4. Process the Text using NLP

In [21]:
def process_text(text, nlp):
    doc = nlp(text)
    return doc

def extract_intent(doc):
    # Basic intent recognition logic (this can be expanded)
    if "weather" in doc.text.lower():
        return "get_weather"
    elif "time" in doc.text.lower():
        return "get_time"
    elif "name" in doc.text.lower():
        return "ask_name"
    elif "stop" in doc.text.lower() or "exit" in doc.text.lower():
        return "stop_listening"
    else:
        return "unknown"


## 5. Execute Actions Based on Intent

In [22]:
def execute_intent(intent):
    if intent == "get_weather":
        return get_weather()
    elif intent == "get_time":
        return get_time()
    elif intent == "ask_name":
        return "I am Jarvis, your AI voice assistant."
    elif intent == "stop_listening":
        return "Stopping now. Goodbye!"
    else:
        return "Sorry, I didn't understand that."

def get_weather():
    api_key = "your_openweather_api_key"  # Replace with your OpenWeather API key
    city = "Chennai"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    weather_data = response.json()
    
    if weather_data["cod"] == 200:
        main = weather_data["main"]
        temperature = main["temp"]
        return f"The current temperature in {city} is {temperature}°C."
    else:
        return "Sorry, I couldn't retrieve the weather information."

def get_time():
    return f"The current time is {time.strftime('%I:%M %p')}."

## 6. Generate and Output Response

In [23]:
def output_response(engine, response_text):
    speak_text(engine, response_text)

## Main Program Loop
def main():
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()
    nlp = spacy.load("en_core_web_sm")
    tts_engine = setup_tts_engine()
    
    start_time = time.time()
    timeout = 60  # Stop listening after 60 seconds

    while True:
        audio = capture_voice_input(recognizer, microphone)
        if audio:
            recognized_text = recognize_speech(recognizer, audio)
            if recognized_text:
                doc = process_text(recognized_text, nlp)
                intent = extract_intent(doc)
                response = execute_intent(intent)
                output_response(tts_engine, response)

                if intent == "stop_listening":
                    break
                
        # Check for timeout
        if time.time() - start_time > timeout:
            print("Timeout reached. Stopping now.")
            break
        
        time.sleep(1)

if __name__ == "__main__":
    main()

Listening...
Google Speech Recognition could not understand audio
Listening...
Recognized: time now
Listening...
Recognized: stop
